In [ ]:
# https://github.com/yunjey/pytorch-tutorial

In [1]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [13]:
# ================================================================== #
#                     1. Basic autograd example 1                    #
# ================================================================== #

# Create tensors.
x = torch.tensor(15., requires_grad=True)
w = torch.tensor(10., requires_grad=True)
b = torch.tensor(900., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
# Computes the sum of gradients of given tensors with respect to graph leaves.
y.backward()

# Print out the gradients.
# Computes and returns the sum of gradients of outputs with respect to the inputs.
print(x.grad)    # x.grad = 2 = dy/dx = w
print(w.grad)    # w.grad = 1 = dy/dw = x
print(b.grad)    # b.grad = 1 = dy/db = 1

tensor(10.)
tensor(15.)
tensor(1.)


In [17]:
# ================================================================== #
#                    2. Basic autograd example 2                     #
# ================================================================== #

# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3) # inputs
y = torch.randn(10, 2) # ouputs
print(x)
print(y)

# Build a fully connected layer.

# Applies a linear transformation to the incoming data: y= xA^T + by
linear = nn.Linear(3, 2) # input size = 3, output size = 2
print ('w: ', linear.weight) # initial values defined here: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear
print ('b: ', linear.bias)

# Build loss function and optimizer.

# Creates a criterion that measures the mean squared error (squared L2 norm) between each element in the input xxx and target yyy.
criterion = nn.MSELoss() 
# Implements stochastic gradient descent (optionally with momentum).
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)
print ('pred: ', pred) # input shape(10,3), output shape (10,2)

# Compute loss.
loss = criterion(pred, y) # using formula from docs
print('loss: ', loss.item()) # cost

# Backward pass.
loss.backward() # run backward pass on output to computer gradients across linear layer

# Print out the gradients.
# calucated from backward pass
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item())

tensor([[-0.6541, -0.4835,  1.2342],
        [-0.3676,  1.2874, -0.3080],
        [-1.1146,  0.1940,  1.0058],
        [ 0.4186, -1.3168,  0.3667],
        [ 0.6491, -1.0791,  0.4444],
        [ 1.8933,  0.0439, -0.2866],
        [-0.3996,  0.9862, -0.7550],
        [ 1.5314,  0.2774,  0.5163],
        [-1.4798, -0.4294, -0.6591],
        [-1.5002, -0.2480, -1.0131]])
tensor([[ 1.6250, -0.7419],
        [ 0.6292, -0.1421],
        [ 0.3259, -0.7808],
        [ 0.3784, -0.4934],
        [ 0.4999, -0.5276],
        [-2.1982,  0.8444],
        [ 1.6091, -0.4208],
        [ 0.6591, -1.3482],
        [ 2.1611, -0.3540],
        [ 0.1025, -0.7208]])
w:  Parameter containing:
tensor([[-0.3922, -0.4607, -0.1982],
        [ 0.3686, -0.2673, -0.2526]], requires_grad=True)
b:  Parameter containing:
tensor([0.5715, 0.0252], requires_grad=True)
pred:  tensor([[ 0.8062, -0.3984],
        [ 0.1836, -0.3766],
        [ 0.7199, -0.6915],
        [ 0.9413,  0.4389],
        [ 0.7260,  0.4406],
        [

In [19]:
# ================================================================== #
#                     3. Loading data from numpy                     #
# ================================================================== #

# Create a numpy array.
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)

# Convert the torch tensor to a numpy array.
z = y.numpy()

# just some conversions
print(x, y, z)

[[1 2]
 [3 4]] tensor([[1, 2],
        [3, 4]]) [[1 2]
 [3 4]]


In [20]:
# ================================================================== #
#                         4. Input pipeline                           #
# ================================================================== #

# Download and construct CIFAR-10 dataset.
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print (image.size())
print (label)

# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

5.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

9.5%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

13.5%IOPub me

In [21]:
# ================================================================== #
#                5. Input pipeline for custom dataset                 #
# ================================================================== #

# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names. 
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open).
        # 2. Preprocess the data (e.g. torchvision.Transform).
        # 3. Return a data pair (e.g. image and label).
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset.
        return 0 

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [22]:
# ================================================================== #
#                        6. Pretrained model                         #
# ================================================================== #

# Download and load the pretrained ResNet-18.
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below.
for param in resnet.parameters():
    param.requires_grad = False

# Replace the top layer for finetuning.
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)
outputs = resnet(images)
print (outputs.size())     # (64, 100)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/willmartin/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%
/Users/willmartin/miniforge3/envs/pytorch/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


torch.Size([64, 100])


In [ ]:
# ================================================================== #
#                      7. Save and load the model                    #
# ================================================================== #

# Save and load the entire model.
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))